In [1]:
import threading
import numpy as np
import ctypes

class Vid(threading.Thread):

    def __init__(self, name):
        threading.Thread.__init__(self)
        self.Valv = 0
        self.name = name
        self.stop = False
    
    def run(self):
        while not self.stop:
            self.Valv = np.random.rand()
            
    def get_id(self): 
  
        # returns id of the respective thread 
        if hasattr(self, '_thread_id'): 
            return self._thread_id 
        for id, thread in threading._active.items(): 
            if thread is self: 
                return id
   
    def raise_exception(self): 
        thread_id = self.get_id()
        print(thread_id)
        res = ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id, ctypes.py_object(SystemExit)) 
        print(res)
        if res > 1: 
            ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id, 0) 
            print('Exception raise failure')

            
vid = Vid('Thread 1')

In [2]:
vid.start()

In [3]:
vid.Valv

0.22699501616458262

In [8]:
vid.is_alive()

False

In [7]:
vid.raise_exception()
vid.join() 

None
0


In [5]:
vid.stop = True

In [ ]:
"""
ADDITION TO THE THREADING TUTORIAL
"""

%%time 

from multiprocessing.dummy import Pool as ThreadPool
def record_torque(args):
    torobo, r = args
    TORQUE = []
    t = 0
    step = r.sleep_dur.to_sec()
    while t < 10:
        TORQUE.append(get_cur_joints_torque(torobo))
        r.sleep()
        t += step
    return np.stack(TORQUE)

pool = ThreadPool(1)
"""
The follwing line blocks code and untill all the workers have returned. Returns a list. 
Because `map` takes a tuple of arguments that it distributes among workers in the ThreadPool.
if there were more than one, we'd get a list of size more that one (one from each worker) 
"""
# sync_result = pool.map(record_torque, ([torobo, r], ))

""" 
this line does not block and runs in the background. You call the line below at any time, but
it will return AFTER the thread has exited.""" 
async_result = pool.apply_async(record_torque, ([torobo, r], )) 
# do something else while the pool (of threads, or just one thread) is running """
# return_val = async_result.get()


In [9]:
"""THIS CELL RUNS EVERYTHING IN A SEPARATE THREAD: SUBSCRIBER THAT STORES VIDEO FRAMES EVERY TIME STEP `IMAGEBUFF`
   AND UPDATES BOUNDING BOXES AROUND THE OBJECT USING CV2 IMSHOW"""

plt.switch_backend('TkAgg')

sys.path.insert(0, '/home/torobo/catkin_ws/src/tutorial/PyTorch-YOLOv3')
from detect_upd import Recog
import random
recog = Recog()

class CV2im(threading.Thread):
    
    def __init__(self):
        s = 'wqerqasdfoiuymnmnbv'
        self.wname = ''.join(random.sample(s,len(s)))
        threading.Thread.__init__(self)
        self.imagebuff = 0
        time.sleep(1)
        self.keepgoing = False
        rospy.Subscriber("/camera/color/image_raw", Image, self.image_callback)
        time.sleep(1)
        self.T = []
        
        
    def image_callback(self, msg):
        self.imagebuff = bridge.imgmsg_to_cv2(msg, "rgb8")

    def stop(self):
        self.keepgoing = False
        
    def get_gaze_offset(self, cx, cy):
        return 640/2 - cx, 480/2 - cy

    def get_gaze_grad(self, offset_x, offset_y):
        ex = 0.2 * np.tanh(0.025*(offset_x))
        ey = 0.2 * np.tanh(0.025*(offset_y))
        return ex, ey

    def get_box_center(self, x1, y1, box_w, box_h):
        return x1 + box_w/2, y1 + box_h/2
    
    def track_obj(self, x,y,w,h):
        global torobo
        cx, cy = self.get_box_center(x, y, w, h)
        gaze_offset = self.get_gaze_offset(cx,cy)
        ex, ey = self.get_gaze_grad(*gaze_offset)
        hor, ver = get_cur_joints(torobo)[0][14:16]
        torobo.move(ToroboOperator.TORSO_HEAD, positions=[0, 0, np.radians(hor)+ex, np.radians(ver)-ey], duration=1)
        
    def run(self):
        self.keepgoing = True
        while True and self.keepgoing:
            detections = recog.detect(self.imagebuff)
            if detections is not None:
                for det in detections:
                    for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
                        if cls_pred == 0:
                            box_w = x2 - x1
                            box_h = y2 - y1
                            image = cv2.rectangle(self.imagebuff, (x1,y1), (x2,y2), (255, 0, 0), 3)
                            x, y, w, h = x1, y1, box_w, box_h
                            self.track_obj(x, y, w, h)
            cv2.imshow(self.wname, cv2.cvtColor(self.imagebuff, cv2.COLOR_RGB2BGR))
            
            k = cv2.waitKey(3) & 0XFF
            if k== 27 :
                break
            self.T.append(time.time())
        cv2.waitKey(1)
        cv2.destroyWindow(self.wname)
        cv2.waitKey(1)

cv2im = CV2im()
cv2im.start()

NameError: name 'plt' is not defined